In [ ]:
# 事前にGCPの認証が必要で、認証方法は環境によって異なる
# colabの場合:
#   セルで下記を実行
#     from google.colab import auth
#     auth.authenticate_user()
#     %env GCLOUD_PROJECT=GCPのプロジェクトID
# PC等のローカル環境の場合:
#   初回のみ、https://cloud.google.com/sdk/docs/install-sdk からgcloud CLIをインストールし、gcloud initを実行
%load_ext google.cloud.bigquery

# 15章 演習問題
## 15-1 多次元分析の前処理
### Q: 予約履歴の多次元分析のためのキューブ作成


In [ ]:
%%bigquery
with
-- （1） reservationから必要なデータを抽出し、customerおよびcampaignと結合
reservation as (
    select
        -- （1）-5 checkout_dateから年と月を抽出
        extract(year from checkout_date at time zone "Asia/Tokyo") as checkout_year,
        extract(month from checkout_date at time zone "Asia/Tokyo") as checkout_month,
        length_of_stay,
        people_num,
        hotel_id,
        sex,
        -- （1）-6 年齢を10刻みでカテゴリ化、60以上は一つのカテゴリにまとめる
        case when age < 60 then floor(age / 10) else 6 end as age_cat,
        campaign_name,
        total_price
    from example.reservation
    -- （1）-3 customer_idをキーとしてcustomerを結合
    left join example.customer using (customer_id)
    -- （1）-4 reserved_atが期間内に入っているcampaignを結合
    left join example.campaign on reserved_at between starts_at and ends_at
    where
        -- （1）-1 キャンセルを除外
        status != "canceled"
        -- （1）-2 2019年のデータを抽出
        and extract(year from checkout_date at time zone "Asia/Tokyo") = 2019
),

-- （2） ディメンションをキーとしてreservationを集約
summary as (
    select
        checkout_year,
        checkout_month,
        hotel_id,
        sex,
        age_cat,
        campaign_name,
        sum(total_price) as sales,
        count(*) as reservation_cnt,
        avg(length_of_stay) as length_of_stay_avg,
        avg(people_num) as people_num_avg
    from reservation
    group by
        checkout_year,
        checkout_month,
        hotel_id,
        sex,
        age_cat,
        campaign_name
),

-- （3） hotelから必要な列を抽出 & unit_priceをカテゴリ化
hotel as (
    select
        hotel_id,
        hotel_name,
        hotel_type,
        address_prefecture,
        address_city,
        case
            when unit_price < 5000 then "1: 0~4999"
            when unit_price < 10000 then "2: 5000~9999"
            when unit_price < 20000 then "3: 10000~19999"
            when unit_price < 30000 then "4: 20000~29999"
            else "5: 30000~"
        end as unit_price_range
    from example.hotel
)

-- （4） 集約データにhotelを結合
select
    checkout_year,
    checkout_month,
    hotel_id,
    hotel_name,
    hotel_type,
    address_prefecture,
    address_city,
    unit_price_range,
    sex,
    age_cat,
    campaign_name,
    sales,
    reservation_cnt,
    length_of_stay_avg,
    people_num_avg
from summary
left join hotel using (hotel_id)

## 15-3 予測モデルの前処理
### Q: 予測モデル作成のためのデータの作成


In [ ]:
%%bigquery
create or replace table example.monthly_customer_feature as
with
--（1）すべての顧客×年月のフレームを準備（初回利用以前を除く）
month_list as (
    select format_date("%Y-%m", tmp_month) as rsv_month
    from unnest(
        generate_date_array("2014-01-01", "2019-12-01", interval 1 month)
    ) as tmp_month
),

existing_customer as (
    select
        customer_id, -- PK
        format_date("%Y-%m", min(reserved_at)) as first_rsv_month
    from example.reservation
    where status != "canceled"
    group by customer_id
),

monthly_customer_frame as (
    select
        rsv_month, -- PK
        customer_id, -- PK
        first_rsv_month
    from existing_customer
    cross join month_list
    where rsv_month >= first_rsv_month
),

--（2）予約履歴を年月ごとに集計
monthly_customer_summary_raw as (
    select
        rsv_month, -- PK
        customer_id, -- PK
        sum(total_price) as total_price,
        count(*) as rsv_cnt,
        count(case when people_num = 1 then 1 end) as rsv_cnt_pnum_1,
        count(case when people_num >= 2 then 1 end) as rsv_cnt_pnum_over2,
        max(reserved_at) as last_rsv_date
    from (
        select
            customer_id,
            format_timestamp("%Y-%m", reserved_at, "Asia/Tokyo") as rsv_month,
            reserved_at,
            people_num,
            total_price
        from example.reservation
        where status != "canceled"
    )
    where
        rsv_month between "2014-01" and "2019-12"
    group by
        rsv_month,
        customer_id
),

--（3） （1）で作成したフレームと（2）で作成した年月ごとの集計を結合して顧客・年月ごとの
--    時系列データを作成
monthly_customer_summary as (
    select
        rsv_month, -- PK
        customer_id, -- PK
        first_rsv_month,
        coalesce(total_price, 0) as total_price,
        coalesce(rsv_cnt, 0) as rsv_cnt,
        coalesce(rsv_cnt_pnum_1, 0) as rsv_cnt_pnum_1,
        coalesce(rsv_cnt_pnum_over2, 0) as rsv_cnt_pnum_over2,
        last_rsv_date
    from monthly_customer_frame
    left join monthly_customer_summary_raw using (rsv_month, customer_id)
),

--（4） 時系列データを加工して特徴量を作成
monthly_customer_feature as (
    select
        rsv_month, -- PK
        customer_id, -- PK
        first_rsv_month,
        sex,
        age,
        coalesce(
            lag(rsv_cnt, 1) over (partition by customer_id order by rsv_month),
            0
        ) as lag_rsv_cnt_m1,
        coalesce(
            lag(rsv_cnt, 2) over (partition by customer_id order by rsv_month),
            0
        ) as lag_rsv_cnt_m2,
        coalesce(
            lag(rsv_cnt, 3) over (partition by customer_id order by rsv_month),
            0
        ) as lag_rsv_cnt_m3,
        sum(rsv_cnt) over (
            partition by customer_id order by rsv_month
            rows between 12 preceding and 1 preceding
        ) as total_rsv_cnt_last_year,
        sum(rsv_cnt_pnum_1) over (
            partition by customer_id order by rsv_month
            rows between 12 preceding and 1 preceding
        ) as total_rsv_cnt_last_year_pnum_1,
        sum(rsv_cnt_pnum_over2) over (
            partition by customer_id order by rsv_month
            rows between 12 preceding and 1 preceding
        ) as total_rsv_cnt_last_year_pnum_over2,
        lag(rsv_cnt, 12) over (partition by customer_id order by rsv_month)
            as lag_rsv_cnt_same_month,
        date_diff(
            parse_date("%Y-%m", rsv_month),
            date(
                last_value(last_rsv_date ignore nulls)
                    over (
                        partition by customer_id order by rsv_month
                        rows between unbounded preceding and 1 preceding
                    ),
                "Asia/Tokyo"
            ),
            day
        ) as diff_days_from_last_rsv_date,
        sum(total_price) over (
            partition by customer_id order by rsv_month
            rows between 12 preceding and 1 preceding
        ) as total_price_last_year,
        case when rsv_cnt >= 1 then 1 else 0 end as rsv_flg
    from monthly_customer_summary
    left join example.customer using (customer_id)
)

--（5）特徴量テーブルを作成（初回利用年月は除く）
select
    rsv_month, -- PK
    customer_id, -- PK
    sex,
    age,
    lag_rsv_cnt_m1,
    lag_rsv_cnt_m2,
    lag_rsv_cnt_m3,
    total_rsv_cnt_last_year,
    total_rsv_cnt_last_year_pnum_1,
    total_rsv_cnt_last_year_pnum_over2,
    lag_rsv_cnt_same_month,
    diff_days_from_last_rsv_date,
    total_price_last_year,
    rsv_flg
from monthly_customer_feature
where rsv_month != first_rsv_month

In [ ]:
%%bigquery
select * from example.monthly_customer_feature